**Умова:**

Є можливість вибрати спосіб виробництва і реалізації двох наборів товарів широкого вжитку. За даними відділу маркетингу, 
яким були проведені дослідження ринку, можливий прибуток від виробництва і реалізації X i Y наведено в таблицях 
(X, Y - прибуток у грошових одиницях):


| X    |  1000  |  1500  | 2000 |
| :--- | :----: | :----: | ---: |
|  P   |  0,5   |  0,3   |  0,2 |      


| Y    |  1000  |  1500  | 1750 |
| :--- | :----: | :----: | ---: |
|  P   |  0,4   |  0,4   |  0,2 |      


Потрібно оцінити ступінь ризику і запропонувати рішення щодо випуску і реалізації одного із наборів товарів.


**Вирішення:**

In [38]:
import pandas as pd
import math

df_x = pd.DataFrame({"X" : [1000, 1500, 2000],
                     "P" : [0.5, 0.3, 0.2]})

df_y = pd.DataFrame({"Y" : [1000, 1500, 1750],
                     "P" : [0.4, 0.4, 0.2]})


**1. Визначення математичного очікування прибутку по кожному проекту:**

In [39]:
df_x["math_value"] = [df_x["X"][i] * df_x["P"][i] for i in range(len(df_x["X"]))]
df_y["math_value"] = [df_y["Y"][i] * df_y["P"][i] for i in range(len(df_y["Y"]))]
EV_x = df_x['math_value'].sum()
EV_y = df_y['math_value'].sum()
print(f" Математичне очікування прибутку по товару X: {EV_x}, a по товвару Y: {EV_y}")

 Математичне очікування прибутку по товару X: 1350.0, a по товвару Y: 1350.0


Очікування прибутку по впровадженню даних груп товарів рівні.




**2. Визначення ступінь ризику при виробництві даних груп товарів.**

Для визначення ступеню ризику, необхідно визначити стандартне відхилення від математичного очікування по кожній групі товару:

In [40]:
df_x["disp_value"] = [((df_x["X"][i] - EV_x)**2)*df_x["P"][i] for i in range(len(df_x["X"]))]
df_y["disp_value"] = [((df_y["Y"][i] - EV_y)**2)*df_y["P"][i] for i in range(len(df_y["Y"]))]
print(f"Стандратне відхилення для групи товарів X: {math.sqrt(df_x['disp_value'].sum())}, а для групи Y: {math.sqrt(df_y['disp_value'].sum())}")

Стандратне відхилення для групи товарів X: 390.51248379533274, а для групи Y: 300.0


**Група товарів Х у випуску і реалізації має більший ризик у зв'язку з більшим відхиленням можливих значень від очікуємого
прибутку. Рекомендую впроваджувати у випуск групу товарів Y.**